In [1]:
!pip install -q kaggle

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!kaggle datasets download -d vuppalaadithyasairam/bone-fracture-detection-using-xrays

 97% 168M/172M [00:06<00:00, 32.9MB/s]
100% 172M/172M [00:06<00:00, 28.7MB/s]


In [5]:
import zipfile
zip_ref=zipfile.ZipFile('/content/bone-fracture-detection-using-xrays.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout, MaxPooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
import os
import cv2
from tensorflow.keras.utils import load_img, img_to_array


In [7]:
DIRECTORY=r'/content/archive (6)'
FOLDERS=['train','val']
CLASSES=['fractured','not fractured']

In [8]:
import os
DIRECTORY=r'/content/archive (6)'
if os.path.exists(DIRECTORY):
  print("Directory exixts.")
else:
  print("Directory does not exits.")

Directory exixts.


In [9]:
X = []
y = []

for classification in CLASSES:
    class_dir = os.path.join(DIRECTORY, 'train', classification)
    images = os.listdir(class_dir)

    for image in images:
        try:
            image_path = os.path.join(class_dir, image)
            if os.path.isfile(image_path):  # Check if the path points to a file
                arr = cv2.imread(image_path)
                if arr is not None:  # Check if the image was loaded successfully
                    arr = cv2.resize(arr, (150, 150))
                    X.append(arr)
                    label = CLASSES.index(classification)
                    y.append(label)
                else:
                    print("Failed to load image:", image_path)
            else:
                print("Not a file:", image_path)
        except Exception as e:
            print("ERROR:", e)

In [10]:
print(len(X))
print(len(y))

8863
8863


In [11]:
X =np.array(X)
y=np.array(y)

In [12]:
print("Shape of X:" ,X.shape)
print("Shape of y:" ,y.shape)

Shape of X: (8863, 150, 150, 3)
Shape of y: (8863,)


In [13]:
X=X/255

In [14]:
from tensorflow.keras.layers import Conv2D ,Dense,Flatten, Dropout,MaxPool2D

In [15]:
model =Sequential()
#Layer1
model.add(Conv2D(64,(3,3),input_shape=(150,150,3),activation='relu'))
model.add(MaxPool2D(2,2))

#Layer2
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.25))

#Layer3
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.25))

#Layer4
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.25))

#FLATTEN
model.add(Flatten())

#DENSE LAYERS
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))

#OUTPUT LAYER
model.add(Dense(2, activation='sigmoid'))

In [16]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 128)      0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 36, 36, 128)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       1

In [1]:
r=model.fit(X,y,epochs=10, batch_size=30,validation_split=0.2)

In [19]:
val_X = []
val_y = []

for feature, label in zip(X, y):
    val_X.append(feature)
    val_y.append(label)

In [1]:
val_X=np.array(val_X)
val_y=np.array(val_y)

In [ ]:
val_X=val_X/255

In [ ]:
test_loss,test_accuracy=model.evaluate(val_X,val_y)

In [ ]:
from sklearn.metrics import accuracy_score,classification_report

In [ ]:
predictions=model.predict(val_X)
predicted_labels=np.argmax(predictions, axix=1)

In [ ]:
accuracy=accuracy_score(val_y, predicted_labels)

In [2]:
print("Test Accuracy: ", accuracy)

In [ ]:
from sklearn.metrics import pair_confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
plt.plot(r.history['Loss'])
plt.plot(r.history['val_loss'])
plt.title('Model Loss Graph')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train','validation'],loc='upper left')
plt.show()

In [ ]:
plt.plot(r.history['accuracy'])
plt.plot(r.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.legend(['train','validation'],loc='upper left')
plt.show()